In [1]:
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from json import loads

In [3]:
tqdm.pandas()

## Scrapping

In [ ]:
# Définition des en-têtes
headers = {"httpapiaccesstoken": "M4rvBxc4M7kqqdtXPDvFEYm9"}
data = {}

# Parcours des IDs avec une barre de progression
for id in tqdm(range(1, 3656), desc="Fetching data"):
    entry = {}  # Dictionnaire pour l'ID actuel
    
    # URL pour route_figures
    url_level = f"https://api.oblyk.org/api/v1/public/crags/{id}/route_figures.json"
    response_level = requests.get(url_level, headers=headers)

    if response_level.status_code == 200:
        try:
            dic_level_response = response_level.json()
            if dic_level_response:
                # Utilisation de get() pour éviter les erreurs si une clé est manquante
                entry["route_count"] = dic_level_response.get("route_count")
                
                # Accès sécurisé aux éléments imbriqués
                grade = dic_level_response.get("grade", {})
                min_grade = grade.get("min", {})
                crag_route = min_grade.get("crag_route")
                if crag_route:  # Vérifiez si crag_route n'est pas None
                    crag = crag_route.get("crag", {})
                    entry["name"] = crag.get("name", "Nom non disponible")
                else:
                    entry["name"] = "Nom non disponible"
                
                entry["climbing_types"] = dic_level_response.get("climbing_types", [])
                entry["levels"] = dic_level_response.get("levels", [])
            else:
                print(f"Aucune donnée trouvée pour ID {id}")
        except ValueError:
            print(f"Invalid JSON for ID {id}")
    
    # URL pour les infos principales
    url_info = f"https://api.oblyk.org/api/v1/public/crags/{id}.json"
    response_info = requests.get(url_info, headers=headers)

    if response_info.status_code == 200:
        try:
            dic_info_response = response_info.json()
            if dic_info_response:
                # Accès sécurisé aux éléments imbriqués
                entry["north"] = dic_info_response.get("north")
                entry["north_east"] = dic_info_response.get("north_east")
                entry["east"] = dic_info_response.get("east")
                entry["south_east"] = dic_info_response.get("south_east")
                entry["south"] = dic_info_response.get("south")
                entry["south_west"] = dic_info_response.get("south_west")
                entry["west"] = dic_info_response.get("west")
                entry["north_west"] = dic_info_response.get("north_west")
                entry["summer"] = dic_info_response.get("summer")
                entry["autumn"] = dic_info_response.get("autumn")
                entry["winter"] = dic_info_response.get("winter")
                entry["spring"] = dic_info_response.get("spring")
                entry["latitude"] = dic_info_response.get("latitude")
                entry["longitude"] = dic_info_response.get("longitude")
                entry["region"] = dic_info_response.get("region")
                entry["rocks"] = dic_info_response.get("rocks")
                entry["photo"] = (
                    dic_info_response.get("photo", {})
                    .get("attachments", {})
                    .get("picture", {})
                    .get("variant_path", "Photo non disponible")
                )
            else:
                print(f"Aucune donnée trouvée pour ID {id}")
        except ValueError:
            print(f"Invalid JSON for ID {id}")
    
    # Ajouter l'ID et les informations associées au dictionnaire principal
    data[id] = entry

# Vérification du résultat
print(f"Nombre total d'entrées collectées : {len(data)}")

In [88]:
df = pd.DataFrame(data)

In [89]:
df = df.T

In [ ]:
# Export CSV
df.to_csv("spots_grimpe.csv", index=False)

## Nettoyage et normalisation des données

In [7]:
# Import CSV
df = pd.read_csv("spots_grimpe.csv")

In [8]:
# Suppression des des lignes qui ont des valeurs nuls dans les colonnes "latitude" ou "longitude"
df = df.dropna(subset=["latitude"])
df = df.dropna(subset=["longitude"])

In [17]:
# Suppression des lignes qui n'ont pas de nom
df = df[df["name"] != "Nom non disponible"]

In [62]:
# Fonction pour extraire les dictionnaires dans la colonne "levels" et créer de nouvelles colonnes avec les clés / valeurs des dictionnaires
def dic_levels(row):
    dico = loads(row["levels"].replace("'",'"'))
    for key, item in dico.items():
        row[key] = item
    return row

In [ ]:
# Apply de la fonction ci-dessus
df = df.apply(dic_levels,axis=1)

In [64]:
# Fonction pour extraire les dictionnaires dans la colonne "climbing_types" et créer de nouvelles colonnes avec les clés / valeurs des dictionnaires
def dic_climbing_types(row):
    dico = loads(row["climbing_types"].replace("'",'"'))
    for key, item in dico.items():
        row[key] = item
    return row

In [65]:
# Apply de la fonction ci-dessus
df = df.apply(dic_climbing_types,axis=1)

In [ ]:
# Suppression des colonnes "Climb_Scrapping" et "levels"
df = df.drop(columns=["climbing_types", "levels"])

In [8]:
# Ajout d'une photo en cas de valeur nul
df["photo"] = df["photo"].fillna("https://i.imghippo.com/files/iRIz5028stI.webp")

In [11]:
# Export CSV
df.to_csv("spots_grimpe.csv", index=False)


In [72]:
df.columns

Index(['route_count', 'name', 'north', 'north_east', 'east', 'south_east',
       'south', 'south_west', 'west', 'north_west', 'summer', 'autumn',
       'winter', 'spring', 'latitude', 'longitude', 'region', 'rocks', 'photo',
       '1a', '1b', '1c', '2a', '2b', '2c', '3a', '3b', '3c', '4a', '4b', '4c',
       '5a', '5b', '5c', '6a', '6b', '6c', '7a', '7b', '7c', '8a', '8b', '8c',
       '9a', '9b', '9c', 'sport_climbing', 'bouldering', 'multi_pitch',
       'trad_climbing', 'aid_climbing', 'deep_water', 'via_ferrata'],
      dtype='object')